# This notebook contains functions to analyse one or several AFC files for a single student

## I. Importation of modules and functions from python scripts

In [ ]:
# We import python modules
# numpy : a mathematical function that is very useful!
import numpy as np
# Pyplot : very important tool if you want to create plots!
import matplotlib.pyplot as plt

# From the module utils we import the function create_dico
# which is helpful if you want to manage files
from utils import create_dico

# From the module analysis we import many functions
from analysis import with_errors, count_errors, count_with_criteria, with_no_errors, count_right

import os

## II. Setting the path to the logfile we are to analyse

In [ ]:
example_directory = "log_files/L2/Jane Doe_Number/logFiles/logFiles"
# The function listdir makes a list of 
# all the files in example_directory
os.listdir(example_directory)

In [ ]:
# We take the directory of a random student, you can change it
example_directory = "log_files/L2/Jane Doe_Number/logFiles/logFiles/"
# example_file = example_directory + "path to the exercise"
example_file = example_directory + "2AFC_i_Mon_Mar_01_17_23_45_2021.log"


## III. Creation of a dictionary

We create a dictionary called *exo_dico*, which will "remember", in a way, the different keys of the *example_file*.

Using the different functions that we imported earlier, we count the number of mistakes (__count_error__), of right answers (__count_right__), of repetitions (__np.sum__) etc. Don't forget that you can always change the key or the criteria you want to count!

In [ ]:
dico_exo = create_dico(example_file)

print(dico_exo.keys())
print(count_errors(dico_exo, key="Vowel", count_one=False))
print(count_right(dico_exo, key="Vowel"))
print(count_errors(dico_exo, key="Stimulus", count_one=False))
print(np.sum(dico_exo['Repetitions']))
print(count_with_criteria(dico_exo, key="Stimulus", criteria='NbErreurs'))
print(np.sum(dico_exo["Response Time"]))


## III. Use of functions to make a simple table

### 1. Right and wrong answers

With the same technique as before we create a dictionary for the exercise.
With the functions that we imported from the module analysis, we count the right and wrong answers and then make a rudimentary table that makes it easier to visualise the statistics for the file.

As always, you can change the key and criteria.

In [ ]:
dico_exo = create_dico(example_file)
print(dico_exo.keys())
err_vowels = count_errors(dico_exo, key="Vowel", count_one=False)
right_vowels = count_right(dico_exo, key="Vowel")
err_stimulus = count_errors(dico_exo, key="Stimulus", count_one=False)
sum_rep = np.sum(dico_exo['Repetitions'])
rep_by_stimulus = count_with_criteria(dico_exo, key="Stimulus", criteria='Repetitions')
sum_time = np.sum(dico_exo["Response Time"])

vowel_keys = set(list(err_vowels.keys()) + list(right_vowels.keys()))
print("key | errors | right")
print("--------------------")
for k in vowel_keys:
    if k in err_vowels.keys():
        error = err_vowels[k]
    else:
        error = 0
    if k in right_vowels.keys():
        right = right_vowels[k]
    else:
        right = 0
    print(f"{k}   |  {error}   | {right}")

You can set a list using the functions. Here we make a list of the vowels for which the student made a mistake, and the ones for which the student made no mistakes. It is not surprising, here, to obtain all the vowels, but feel free to play around with it.

In [ ]:
set(list(err_vowels.keys()) + list(right_vowels.keys()))


### 2. Right and wrong answers and response time

We create a dictionary for the same file but this time with three attributes: the sum of errors (__count_errors__), the sum of right answers (__count_right__), and the total response time (__count_with_criteria__). The dictionaries for these functions are called *err_dico*, *right_dico*, and *time_dico*, respectively.

With the function __time_counters__ we count the number of times we encounter a specific key so that we can devide the total by the result of __time_counters__ and obtain the average.

The function __print_stat_exos__ displays a simple table.

In [ ]:
def exo_stat(file, key="Vowel"):
    dico_exo = create_dico(file)
    err_dico = count_errors(dico_exo, key=key, count_one=False)
    right_dico = count_right(dico_exo, key=key)
    time_dico = count_with_criteria(dico_exo, key=key, criteria='Response Time')

    keys = set(list(err_dico.keys()) + list(right_dico.keys()))
    res = {}
    # we create a new empty dictionary called time_counters
    time_counters = {}
    # for the variable "k" in keys
    for k in keys:
        # if it is the first time we encounter it
        # the time_counters will be set at value 0
        time_counters[k] = 0
    for k in keys:
        # if we already encountered k in the dico of errors
        if k in err_dico.keys():
            # then errors will take the value of k
            error = err_dico[k]
        else:
            # else, error takes the value 0
            error = 0
        if k in right_dico.keys():
            right = right_dico[k]
        else:
            right = 0
        if k in time_dico.keys():
            time = time_dico[k]
            # We increase the counter by increments
            time_counters[k] += 1
        else:
            time = 0
        # the dictionary res that we created earlier
        # will take the values of error, right and time.
        res[k] = [error, right, time]
    # We calculate the average for each key
    for k in keys:
        # we divide the total by the number of occurrences.
        res[k][2] /= time_counters[k]
    return res

def stat_mult_exo(list_files, key="Vowel"):
    # key : [attribute 1, attribute 2]
    # attribute 1 = sum of wrong answers,
    # attribute 2 = sum of right answers
    dico_res = {}
    # for any file in the list of files
    # we provide to the function,
    for file in list_files:
        # we retrieve the statistics for an exercise
        dico_exo = exo_stat(file, key=key)
        # We combine the data
        for k, v in dico_exo.items():
            # If we have already encountered this key, we add the value
            if k in dico_res.keys():
                # we combine the first attribute
                dico_res[k][0] += dico_exo[k][0]
                # we combine the second attribute
                dico_res[k][1] += dico_exo[k][1]
            else:
                # If not, we set up the values corresponding
                # to the key for the current exercise
                dico_res[k] = dico_exo[k]
    return dico_res

# We create a rudimentary table to display the results.
def print_stat_exo(dico_exo):
    print("| key  | errors | right |      resp time      |")
    print("|---------------------------------------------|")
    for k, v in dico_exo.items():
        print(f"| {k}    |   {v[0]}  |   {v[1]}  | {v[2]}  |")

In [ ]:
dico_multi = stat_mult_exo([example_file], key="Vowel")
print_stat_exo(dico_multi)

## IV. Selecting files that have the pattern "AFC" to analyse them

### 1. Definitions of functions

The function __get_files_directory__ has two attributes that are *directory* and *pattern = "AFC"*. The purpose of the function is to add a backslash if the *example_directory* does not have one at the end, it avoids the raise of errors. 

The previous function also makes a list of the files that have the demanded pattern (here "AFC"), and this list is used in the function __get_files_mult_directory__.

In [ ]:
example_directory = "log_files/L2/Jane Doe_Number/logFiles/logFiles/"

# We retrieve the files with the pattern "AFC"
def get_files_directory(directory, pattern="AFC"):
    # if the last element of the directory is not "/"
    if directory[-1] != "/":
        # then, we add it.
        directory += "/"
    # we do it only on file that gave the demanded 
    # pattern, if they are in the directory
    return [directory + file for file in os.listdir(directory) if  pattern in file]

# Now we try to broaden our scope and analyse more 
# than one file at the time. To do so, we compile the
# precedent function to this one and eturn a list of
# files which have one thing in common: their pattern.
def get_files_mult_directory(directory_list, pattern="AFC"):
    res = []
    for d in directory_list:
        res.extend(get_files_directory(d, pattern=pattern))
    return res


### 2. Application of the previously introduced functions

The following cell is the application of the functions that have been defined previously.

In this sense, we take the path to a directory that contains a certain number of files and we make a table which sums the different parameters.

Feel free to change the parameters.

In [ ]:
directory_list = ["log_files/L2/Jane Doe_Number/logFiles/logFiles/"]

AFC_files = get_files_mult_directory(directory_list, pattern="AFC")
dico_multi = stat_mult_exo(AFC_files, key="Vowel")
print_stat_exo(dico_multi)